<a href="https://colab.research.google.com/github/danielbauer1979/MSDIA_PredictiveModelingAndMachineLearning/blob/main/GB886_VII_9_Prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Univariate Time Series

This codebook goes through a few time series examples to showcase basics of univariate time series modeling for generating forecasts. We first look at some basic examples where we carry the steps out manually. We then show how packages and particularly Meta's Prophet can automate the analysis.


As usually, let's load relevant libraries where we rely on time series functionality in statsmodels for our analysis

In [ ]:
# Standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Time series functionality
from statsmodels.tsa.stattools import adfuller #ADF test for stationarity
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA

#Prophen by Meta, https://facebook.github.io/prophet
from prophet import Prophet

#For calculating error metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error

## Two Examples

We will go through two introductory examples to demonstrate different types of time series: We look at the S&P500 stock index (daily observations) as one example, and an example of retail sales (monthly observations) as a second example.

### Retail Sales Data

Let's load the data:

In [ ]:
!git clone https://github.com/danielbauer1979/MSDIA_PredictiveModelingAndMachineLearning.git

In [ ]:
dat_sales = pd.read_csv('MSDIA_PredictiveModelingAndMachineLearning/GB886_VII_3_RetailSales.csv')
dat_sales.tail()

The data goes from January 1992 (index zero) to May 2016 (last index). Let's plot it:

In [ ]:
plt.plot(dat_sales)
plt.title('Retail Sales')
plt.show()

It is from **visual inspection** [link text](https://)evident that there is seasonality---the same pattern seems to occur over and over again. This is no suprise for retail sales (increased sales around the holidays, say), so our intuition is also helpful. Furthermore, there seems to be a time trend (the series trends upwards).

So, to remove the trend, we use the "decomposition" functionality in statsmodels:

In [ ]:
# Perform seasonal decomposition
result = seasonal_decompose(dat_sales, model='additive', period=12)

# Plot the trend component
plt.plot(result.trend)
plt.title('Trend Component of Retail Sales')
plt.show()

#Plot the seasonal component
plt.plot(result.seasonal)
plt.title('Seasonal Component of Retail Sales')
plt.show()

# Plot the residual component
plt.plot(result.resid)
plt.title('Residual Component of Retail Sales')
plt.show()

We notice the clear trend and the seasonal pattern (which keeps on repeating). Let's check whether the residual is stationary using the *Augmented Dickey Fuller (ADF) test*, where the Null hypothesis is *non-stationarity*---so, if the p-value is small and we reject, there is evidence for stationarity:

In [ ]:
result_adf = adfuller(result.resid.dropna())
print('ADF Statistic: %f' % result_adf[0])
print('p-value: %f' % result_adf[1])
print('Critical Values:')
for key, value in result_adf[4].items():
	print('\t%s: %.3f' % (key, value))


So, it appears that the ADF test rejects, so it seems after removing trend and seasonality, the data appears stationary!

Let's generate the autocorrelation function (ACF) and the partial autocorrelation function (PACF). As reminder, under the Box-Jenkins approach, inpsection of the ACF and the PACF are key to modeling the stationary part.

In [ ]:
plot_acf(result.resid.dropna(), lags=20)
plt.title('ACF of Residuals')
plt.show()

plot_pacf(result.resid.dropna(), lags=20)
plt.title('PACF of Residuals')
plt.show()

So, we see some significant spikes in both the ACF and the PACF until lag 12. the spikes in the PACF seem more subsantive. There are several models that may be possible. We could opt for an ARMA(12,12), which is appropriate for decays after 12 lags on both ACF and PACF. Another option may be an AR(12) because we see significant spikes in the PACF.

Let's check both options:

In [ ]:
model_12_12 = ARIMA(result.resid.dropna(), order=(12, 0, 12))
results_12_12 = model_12_12.fit()
print(results_12_12.summary())

We get a convegrence warning, saying that the covariance matrix is close to singular. This means we should be careful in interpreting the standard errors, but we will ignore the convergence warnings for now.

In [ ]:
model_ar12 = ARIMA(result.resid.dropna(), order=(12, 0, 0))
results_ar12 = model_ar12.fit()
print(results_ar12.summary())

The Ljung-Box Q test that checks for any remaining autocorrelation rules out remaining auto-correlation in the ARMA(12,12) model (the metric is zero) but there still seems to be some remaining autocorrelation on the AR(12). However, the JB test suggest that the AR(12) residuals look more Normal.

Let's run with the ARMA(12,12) model. To validate, let's look at the residuals:

In [ ]:
residuals_12_12 = results_12_12.resid

# Plot the residuals
plt.plot(residuals_12_12)
plt.title('Residuals of ARMA(12, 12) Model')
plt.show()

# Plot the histogram of residuals
plt.hist(residuals_12_12)
plt.title('Histogram of Residuals')
plt.show()

# Plot the ACF of residuals
plot_acf(residuals_12_12, lags=20)
plt.title('ACF of Residuals (ARMA(12, 12))')
plt.show()

# Plot the PACF of residuals
plot_pacf(residuals_12_12, lags=20)
plt.title('PACF of Residuals (ARMA(12, 12))')
plt.show()


So they look reasonably like **white noise**: The realizations look roughly Normal and the ACF and PACF don't show any residual autocorrelations.

Let's appraise the **performance of the forecasts** using this model. In doing so, we split the full time seroes into two windows: the last 24 data points as the validation set and then the remaining data points as the training data set. We will use the training set to fit our model, and then check how well it performs relative to our validation data (however, before we use the model in our applications, we would (re-)estimate it on the full data as we did above):

In [ ]:
train_data = dat_sales[:-24]
validation_data = dat_sales[-24:]

Let's obtain the trend and seasonality component by decomposing the train_data. And then we fit our ARMA(12,12) model on the residuals of train_data:

In [ ]:
decomp_train = seasonal_decompose(train_data, model='additive', period=12, extrapolate_trend='freq')

armamodel_train = ARIMA(decomp_train.resid.dropna(), order=(12, 0, 12))
fittedarma_train = armamodel_train.fit()

We will forecast the trend component by using the difference between the most recent trend components and extrapolating that to the future (i.e., $trend(T+\tau)=trend(T-1) + (\tau+1) \times [ trend(T) - trend(T-1)]$).

In [ ]:
forecast_trend = decomp_train.trend.iloc[-1] + np.arange(1, 25) * decomp_train.trend.diff().iloc[-1]

And we just use the past 12 monyhs to forecast for the next 24 months (we repeat twice):

In [ ]:
forecast_seasonal = decomp_train.seasonal[-12:].values
forecast_seasonal = np.tile(forecast_seasonal, 2)

Finally, we forecast our ARMA(12,12) model and combine the three components plus the confidence interval from the ARMA model to get our total forecasts:

In [ ]:
forecast_arma = fittedarma_train.predict(start=len(decomp_train.resid.dropna()), end=len(decomp_train.resid.dropna()) + 23)
forecast = forecast_trend + forecast_seasonal[:24] + forecast_arma.values

forecast_arma_ci = fittedarma_train.get_forecast(steps=24).conf_int()
forecast_lower = forecast_trend + forecast_seasonal[:24] + forecast_arma_ci.iloc[:, 0].values
forecast_upper = forecast_trend + forecast_seasonal[:24] + forecast_arma_ci.iloc[:, 1].values

# Create a DataFrame for the forecast with confidence intervals
forecast_df = pd.DataFrame({'Forecast': forecast,
                           'Lower Bound': forecast_lower,
                           'Upper Bound': forecast_upper},
                          index=validation_data.index)

Here are our forecasts:

In [ ]:
forecast_df['Actual'] = validation_data.values

In [ ]:
forecast_df

We can see that the confidence inteval seems to include the actuals most of the time, and that the forecasts are close to the actuals!

Let's generate a plot of the series, the validation set, our forecasts, and the validation data:

In [ ]:
plt.plot(train_data, label='Training Data')
plt.plot(validation_data, label='Actual Values')
plt.plot(forecast_df['Forecast'], label='Forecast')
plt.fill_between(forecast_df.index, forecast_df['Lower Bound'], forecast_df['Upper Bound'], color='gray', alpha=0.6)
plt.legend()
plt.title('Retail Sales Forecast')
plt.show()

And, finally, let's calculate some error metrics:

In [ ]:
# Calculate RMSE
rmse = np.sqrt(mean_squared_error(validation_data, forecast_df['Forecast']))
print('RMSE:', rmse)

# Calculate MAE
mae = mean_absolute_error(validation_data, forecast_df['Forecast'])
print('MAE:', mae)

# Calculate MAPE
mape = np.mean(np.abs((validation_data.values.flatten() - forecast_df['Forecast'].values))/validation_data.values.flatten()) * 100
print('MAPE:', mape)


Pretty good!

As an alternative, we consider using Facebook's **Prophet** Time Series toolbox for the same data. The data need to be in a specific format, with date ('ds') and variable ('y') columns. We can obtain this from the Prophet github site, where we got this data from in the first place:

In [ ]:
dat_sales = pd.read_csv('https://raw.githubusercontent.com/facebook/prophet/main/examples/example_retail_sales.csv')
dat_sales

We will model the data as a time series object in Prophet:

In [ ]:
time_series = Prophet()
time_series.fit(dat_sales)

And, to start with, we will forecast over the next 24 months:

In [ ]:
future = time_series.make_future_dataframe(periods=24,freq='MS')
future.tail()

Let's generate predictions:

In [ ]:
forecast = time_series.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

These seem fairly similar to our "manual" predictions. Let's visualize:

In [ ]:
prophet_fig_1 = time_series.plot(forecast)

And the components:

In [ ]:
prophet_fig_2 = time_series.plot_components(forecast)

To illustrate how these forecasts came about, let's look into some of the detail. For instance, let's look at the change points that are incorporated in the trend component:

In [ ]:
from prophet.plot import add_changepoints_to_plot
fig = time_series.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), time_series, forecast)

So we notice a number of change points that are included.

And let's check the preformance by evalualting some out-of-sample error metrics. We will use cross validation, which is automated in Prophet (although Prophet operates using daily data):

In [ ]:
from prophet.diagnostics import cross_validation
df_cv = cross_validation(time_series, initial='1095 days', period='365 days', horizon = '730 days')
#You can adjust the windows---here we use three years initially and then various windows thereafter

Let's investigate the perfomance:

In [ ]:
from prophet.diagnostics import performance_metrics
df_p = performance_metrics(df_cv)
df_p.tail()

So using the various windows, we notice that the errors are a bit larger than in our validation using the single hold-out window. Let's visualize:

In [ ]:
from prophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(df_cv, metric='mape')

So the further we forecast, the larger the error---which makes sense. But a MAPE of 10% is still pretty good.

### S&P 500 Daily Data

Let's now load and look at the stock data:

In [ ]:
dat_sandp = pd.read_csv('MSDIA_PredictiveModelingAndMachineLearning/GB886_VII_3_SanP500.csv')
dat_sandp.head()

The data is from 2/28/2023 (index zero) until 2/28/2024 (last index). Let's take a quick look:

In [ ]:
plt.plot(dat_sandp)
plt.title('Evolution of S and P Index 3/2023 to 3/2024')
plt.show()

So, the first question is to check whether the time series is stationary. Here, **visual inspection** again makes it clear that the data appears not stationary, although there is also not a very clear deterministic trend.

Let's check the ADF test:

In [ ]:
result = adfuller(dat_sandp)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))


So, here the ADF test does not reject so we **do not** have stationarity. So, let's take the **first differences**, i.e., let's look at
$$
y^d_t = y_t - y_{t-1}
$$
and then work with the differenced series $\{y^d_1,y^d_2,y^d_3,...\}$:

In [ ]:
dat_sandp_diff = dat_sandp.diff().dropna()
plt.plot(dat_sandp_diff)
plt.title('First Differences of S and P Index 3/2023 to 3/2024')
plt.show()

So, there is no longer an indication of a trend. Let's run the ADF test on the differenced series.

In [ ]:
result = adfuller(dat_sandp_diff)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

So, that looks stationary. So, we can continue with our anaylsis using the differenced series.

Let's plot the ACF and the PACF:

In [ ]:
plot_acf(dat_sandp_diff, lags=20)
plt.title('ACF of Differenced Series')
plt.show()

plot_pacf(dat_sandp_diff, lags=20)
plt.title('PACF of Differenced Series')
plt.show()

So we don't detect much correlation or autocorrelation. Let's look at a histogram:

In [ ]:
plt.hist(dat_sandp_diff)
plt.title('Histogram of Differenced Series')
plt.show()

So, all in all, it seems reasonable to assume that the series is white noise. Let's look at the mean and variance:

In [ ]:
mean_diff = dat_sandp_diff.mean()
std_diff = dat_sandp_diff.std()

# Calculate the standard error of the mean
n = len(dat_sandp_diff)
se_mean = std_diff / np.sqrt(n)

# Calculate the 95% confidence interval for the mean
lower_mean = mean_diff - 1.96 * se_mean
upper_mean = mean_diff + 1.96 * se_mean

print("Point Estimate for Mean:", mean_diff.iloc[0])
print("95% Confidence Interval for Mean:", (lower_mean.iloc[0], upper_mean.iloc[0]))
print("Standard Deviation:", std_diff.iloc[0])

So, the mean is significantly different from zero. Our final model for the series is:
$$
S_t = S_{t-1} + 4.45836 + 33.8281 \times \varepsilon_t,\;\;\varepsilon_t\sim N(0,1)
$$
Such a time series is called a **random walk with drift**.

Let's **simulate** some realizations going forward. When simulating, i.e., drawing random realizations, it is always a good idea to set a so-called *random seed* to make the results reproducable. That is, the next time we run the simulation, we want to draw the same random numbers:

In [ ]:
np.random.seed(86)

We will generate 20 simulated paths starting at the endnpoint, where we will similate 250 days forward. We use our random walk with drift model for the simulation:

In [ ]:
num_simulations = 20
num_steps = 250

simulated_series = []
for j in range(num_simulations):
  series = [dat_sandp.iloc[-1, 0]]  # Start with the last value of the original series
  for j in range(num_steps):
    drift = mean_diff.iloc[0]
    shock = np.random.normal(0, std_diff.iloc[0])
    new_value = series[-1] + drift + shock
    series.append(new_value)
  simulated_series.append(series)

Let's visualize by plotting our original series and then appending our different **scenarios** at the end:



In [ ]:
plt.plot(dat_sandp.index, dat_sandp.values, label='Original Series')

for series in simulated_series:
  plt.plot(np.arange(dat_sandp.index[-1]+1, dat_sandp.index[-1]+1+len(series)), series, alpha=0.5)

plt.title('Simulated Random Walks with Drift')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.show()

So, we see under this model the possible values for the S&P 500 going forward are fairly wide!